In [23]:
!pip install folium
!pip install matplotlib folium networkx pandas



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [24]:
import folium
import networkx as nx
import pandas as pd
from matplotlib import cm, colors

In [22]:
node_data = pd.read_csv("../data/node1.csv")
link_data = pd.read_csv("../data/link1.csv")

nodes = []
links = []

for _, n in node_data.iterrows():


    node_dict = {}
    node_dict["node_id"] = n["node_id"]
    node_dict["lat"] = n["lat"]
    node_dict["lon"] = n["lon"]
    nodes.append(node_dict)

for _, l in link_data.iterrows():

    link_dict = {}
    link_dict["source"] = l["start_id"]
    link_dict["target"] = l["end_id"]
    link_dict["weight"] = l["width"]

    links.append(link_dict)


G = nx.Graph()

for node in nodes:
    G.add_node(node["node_id"],lat=node["lat"],lon=node["lon"])

# エッジを追加
for link in links:
    G.add_edge(link["source"], link["target"], weight=link["weight"])


# 地図の中心を設定
center_lat = sum(node["lat"] for node in nodes) / len(nodes)
center_lon = sum(node["lon"] for node in nodes) / len(nodes)
m = folium.Map(location=[center_lat, center_lon], zoom_start=17)



# エッジを地図に追加
for source, target,edge_data in G.edges(data=True):
    source_data = G.nodes[source]
    target_data = G.nodes[target]
    edge_weight = edge_data["weight"]  # エッジの重み
    folium.PolyLine(
        locations=[
            [source_data["lat"], source_data["lon"]],
            [target_data["lat"], target_data["lon"]]
        ],
        color="red",
        weight=edge_weight * 3,  # 重みを太さに反映
        opacity=0.6
    ).add_to(m)

# 地図を保存
m.save("network_map_with_networkx.html")


In [ ]:
node_data = pd.read_csv("../data/node.csv")
link_data = pd.read_csv("../data/link.csv")

nodes = []
links = []

for _, n in node_data.iterrows():


    node_dict = {}
    node_dict["node_id"] = n["node_id"]
    node_dict["lat"] = n["lat"]
    node_dict["lon"] = n["lon"]
    nodes.append(node_dict)

for _, l in link_data.iterrows():

    link_dict = {}
    link_dict["source"] = l["start_id"]
    link_dict["target"] = l["end_id"]
    link_dict["weight"] = l["width"]
    link_dict["vtcl_slope"] = l["vtcl_slope"]
    link_dict["lev_diff"] = l["lev_diff"]

    links.append(link_dict)


G = nx.Graph()

for node in nodes:
    G.add_node(node["node_id"],lat=node["lat"],lon=node["lon"])

# エッジを追加
for link in links:
    G.add_edge(link["source"], link["target"], weight=link["weight"],slope=link["vtcl_slope"],diff=link["lev_diff"] )


# 地図の中心を設定
center_lat = sum(node["lat"] for node in nodes) / len(nodes)
center_lon = sum(node["lon"] for node in nodes) / len(nodes)
m = folium.Map(location=[center_lat, center_lon], zoom_start=17)


# 値（easily）の範囲を取得

easily = [G.edges[edge]["slope"]* G.edges[edge]["diff"]for edge in G.edges]

min_weight = min(easily)
max_weight = max(easily)
print(max_weight,min_weight)
norm = colors.Normalize(vmin=min_weight, vmax=max_weight)
cmap = cm.get_cmap("coolwarm")  # 赤から青

# エッジを地図に追加
for source, target,edge_data in G.edges(data=True):
    source_data = G.nodes[source]
    target_data = G.nodes[target]
    edge_weight = edge_data["weight"]  # エッジの重み
    slope = edge_data["slope"]
    diff = edge_data["diff"]
    colormap = slope*diff
    rgba_color = cmap(norm(colormap))  # 正規化してカラー取得
    hex_color = colors.rgb2hex(rgba_color)  # Hexカラーに変換

    folium.PolyLine(
        locations=[
            [source_data["lat"], source_data["lon"]],
            [target_data["lat"], target_data["lon"]]
        ],
        color=hex_color,
        weight=edge_weight * 3,  # 重みを太さに反映
        opacity=0.6
    ).add_to(m)

# 地図を保存
m.save("network_map_with_networkx.html")

6 1


/var/folders/r2/q8_hcldj4f33gjgpcmt2s_yw0000gn/T/ipykernel_73741/3415620833.py:52: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("coolwarm")  # 赤から青
